In [69]:
import keras
from keras.layers import Activation, BatchNormalization
from keras.layers import Conv2D, Dense, Flatten, Input
from keras.models import Model
import numpy as np
import time



class smallNN:
    def __init__(self):
        #self.board_input = Input(shape=TrojanGoPlane.shape(), name='board_input')
        self.board_input = Input(shape=(7, 5, 5), name='board_input')

    def nn_model(self):

        pb = self.board_input
        for i in range(4):                     # <1>
            pb = Conv2D(64, (3, 3),        # <1>
                padding='same',                # <1>
                data_format='channels_first',  # <1>
                activation='relu')(pb)     # <1>



        policy_conv = \
            Conv2D(2, (1, 1),                          # <2>
                data_format='channels_first',          # <2>
                activation='relu')(pb)                 # <2>
        policy_flat = Flatten()(policy_conv)           # <2>
        policy_output = \
            Dense(26,
                  activation='softmax')(policy_flat)   # <2>




        value_conv = \
            Conv2D(1, (1, 1),                                    # <3>
                data_format='channels_first',                    # <3>
                activation='relu')(pb)                           # <3>
        value_flat = Flatten()(value_conv)                       # <3>
        value_hidden = Dense(256, activation='relu')(value_flat) # <3>
        value_output = Dense(1, activation='tanh')(value_hidden) # <3>

        model = Model(
            inputs=[self.board_input],
            outputs=[policy_output, value_output])

        return model



class trojanGoZero:
    def __init__(self, num_resnet_block=19):
        #self.board_input = Input(shape=TrojanGoPlane.shape(), name='board_input')
        self.board_input = Input(shape=(7, 5, 5), name='board_input')
        self.num_resnet_block = num_resnet_block
        self.num_filters = 256

    
    def resNetBlock(self, x,filters,pool=False):
        res = x
        
        if pool:
            x = MaxPooling2D(pool_size=(2, 2))(x)
            res = Conv2D(filters=filters,kernel_size=[1,1],strides=(2,2),padding="same", data_format='channels_first')(res)
        #out = BatchNormalization()(x)
        #out = Activation("relu")(out)
            
        out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding="same", data_format='channels_first')(x)
        out = BatchNormalization()(out)
        out = Activation("relu")(out)
        
        out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding="same", data_format='channels_first')(out)
        out = BatchNormalization()(out)
        
        out = keras.layers.add([res,out])
        out = Activation("relu")(out)

        return out


        
    def nn_model(self, input_shape):
        #Input feature of 17*19*19 or 7*5*5 as board_input or board_images
        
        board_images = Input(input_shape)
        #board_images = self.board_input

        #CNN-1 with Batch Normalization and rectifier nonlinearity.
        cnn1 = Conv2D(filters=256, kernel_size=[3, 3], strides=[1, 1], padding="same", data_format='channels_first')(board_images)
        cnn1_batch = BatchNormalization()(cnn1)
        cnn1_act = Activation("relu")(cnn1_batch)

        self_in = cnn1_act
        
        #Now build 19 or 39 ResNet block networks depends on "num_resnet_block" variable.
        for i in range(self.num_resnet_block):
            self_out = self.resNetBlock(self_in, self.num_filters)
            self_in = self_out

        out = self_out
        
        policy_conv = \
            Conv2D(2, (1, 1),                          # <2>
                data_format='channels_first',          # <2>
                activation='relu')(out)                # <2>
        policy_conv_bn = BatchNormalization()(policy_conv)
        policy_flat = Flatten()(policy_conv_bn)           # <2>
        policy_output = \
            Dense(26,
                  activation='softmax')(policy_flat)   # <2>




        value_conv = \
            Conv2D(1, (1, 1),                                    # <3>
                data_format='channels_first',                    # <3>
                activation='relu')(out)                          # <3>
        value_conv_bn = BatchNormalization()(value_conv)
        value_flat = Flatten()(value_conv_bn)                       # <3>
        value_hidden = Dense(256, activation='relu')(value_flat) # <3>
        value_output = Dense(1, activation='tanh')(value_hidden) # <3>

        model = Model(
            inputs=[board_images],
            outputs=[policy_output, value_output])

        return model


In [70]:
net = trojanGoZero()
input_shape = (7,5,5)
model = net.nn_model(input_shape)
print(model.summary())
    
  

net_small = smallNN()
model_small = net_small.nn_model()
print(model_small.summary())
    

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 7, 5, 5)      0                                            
__________________________________________________________________________________________________
conv2d_48 (Conv2D)              (None, 256, 5, 5)    16384       input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_42 (BatchNo (None, 256, 5, 5)    20          conv2d_48[0][0]                  
__________________________________________________________________________________________________
activation_40 (Activation)      (None, 256, 5, 5)    0           batch_normalization_42[0][0]     
____________________________________________________________________________________________

dense_10 (Dense)                (None, 26)           1326        flatten_7[0][0]                  
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 1)            257         dense_11[0][0]                   
Total params: 123,314
Trainable params: 123,314
Non-trainable params: 0
__________________________________________________________________________________________________
None


In [49]:
#use HDF5 to store and load
import h5py


class ExperienceBuffer:
    def __init__(self, model_input, action_target, value_target):
        self.model_input = model_input
        self.action_target = action_target
        self.value_target = value_target
        
    def serialize(self, h5file):
        h5file.create_group('experience')
        h5file['experience'].create_dataset('model_input', data=self.model_input)
        h5file['experience'].create_dataset('action_target', data=self.action_target)
        h5file['experience'].create_dataset('value_target', data=self.value_target)

    
def load_experience(h5file):
    return ExperienceBuffer(model_input=np.array(h5file['experience']['model_input']),
                            action_target=np.array(h5file['experience']['action_target']),
                            value_target=np.array(h5file['experience']['value_target'])
                            )


start = time.time() 
with h5py.File('experience_1.hdf5', 'r') as exp_input:
    experience_buffer = load_experience(exp_input)

finish = time.time()
print("Read Time taken :", finish - start)     

Read Time taken : 0.008195877075195312


In [50]:
len(experience_buffer.value_target)

62

In [51]:
experience_buffer.action_target[10]

array([0.0625, 0.    , 0.0625, 0.    , 0.    , 0.    , 0.    , 0.0625,
       0.    , 0.0625, 0.0625, 0.0625, 0.    , 0.0625, 0.0625, 0.0625,
       0.    , 0.    , 0.0625, 0.0625, 0.0625, 0.    , 0.0625, 0.0625,
       0.0625, 0.0625])

In [52]:
type(experience_buffer.action_target)

numpy.ndarray

In [53]:
len(experience_buffer.value_target)

62

In [54]:
experience_buffer.model_input.shape[0]

62

In [26]:
import enum
from collections import namedtuple
import operator

class Player(enum.Enum):
    black = 1
    white = 2

    @property
    def opp(self):  #opposition
        if self == Player.white:
            return Player.black 
        else:
            return Player.white


In [29]:
print(Player.black,  "PASS")

Player.black PASS


In [42]:
iteration = 2
for i in range(iteration):
    filename = "./data/experience_" + str(i) + ".hdf5"
    print(filename)

./data/experience_0.hdf5
./data/experience_1.hdf5


In [67]:
filename = "./data/experience_0.hdf5"
#filename = filename.split("/")
filename = filename.split("/")[-1].split(".")[0]

In [68]:

model_name = 'checkpoints/iteration_Savedmodel/' + filename + '_model'
model_name

'checkpoints/iteration_Savedmodel/experience_0_model'